In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{num}의 10배는?")
prompt

PromptTemplate(input_variables=['num'], template='{num}의 10배는?')

In [4]:
from langchain_openai import ChatOpenAI

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")
chain.invoke({"num" : 5})

AIMessage(content='50', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3f1d01bb-86fd-464f-b5f4-a7ff036fcb6d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

### 데이터 효과적 전달법
- RunnablePasthrough 는 입력 변경하지 않거나 추가 키를 더하여 전달 가능 
- RunnableParallel과 함께 사용, 새 키에 데이터 할당하는데 사용 

In [5]:
from langchain_core.runnables import RunnablePassthrough

In [6]:
chain.invoke({"num": 2})

AIMessage(content='20입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 15, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f0d9f26d-3101-4083-b4e4-47d3d0f43468-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18})

In [7]:
runnable_chain = {"num" : RunnablePassthrough()} | prompt | ChatOpenAI()
runnable_chain.invoke(10)

AIMessage(content='100입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 15, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6766ef1d-d6c4-4321-8de8-edd18039b09c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18})

In [8]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

In [9]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

In [14]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [11]:
chain1 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 10배는?\n답변(결과만): ")
    | ChatOpenAI()
)
chain2 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 1/10배는?\n답변(결과만): ")
    | ChatOpenAI()
)


In [12]:
combined_chain = RunnableParallel(a=chain1, b=chain2)
combined_chain.invoke({"num": 10})


{'a': AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 31, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b56c6b93-fbc4-4bb0-b911-2f8d40b75ad7-0', usage_metadata={'input_tokens': 31, 'output_tokens': 1, 'total_tokens': 32}),
 'b': AIMessage(content='1', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 33, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5d906c9-a36d-4a2f-8c7d-436a088bfada-0', usage_metadata={'input_tokens': 33, 'output_tokens': 1, 'total_tokens': 34})}

In [16]:
# 사용자 정의 함수 매핑 
from langchain_core.runnables import RunnableLambda

def extra(x):
    ext = x["extra"]
    return int(ext["num"]) * int(ext["mult"])

(runnable | RunnableLambda(extra)).invoke({"num": 3})

27